In [1]:
import os
from raphtory import Graph

# Specify the file path
binecode_path = "C:/ms_bincode/Graph_3"

# Load the file
if os.path.isfile(binecode_path):
    g = Graph.load_from_file(binecode_path)
    print(g)
else:
    print(f"File does not exist: {binecode_path}")

Graph(number_of_edges=97252, number_of_vertices=26533, number_of_temporal_edges=159288593, earliest_time="7200000", latest_time="10799999")


In [5]:
from raphtory import algorithms as rp
from raphtory import Graph

start_time = 10200000 # Minute 170
end_time = 10200000 +60000 # Minute 171

g_spike_spike = g.window(start_time, end_time)
print(g_spike_spike)

Graph(number_of_edges=34081, number_of_vertices=12182, number_of_temporal_edges=3273937, earliest_time="10200000", latest_time="10259999")


In [6]:
hits = rp.hits(g_spike_spike)
df = hits.to_df()
print(df)

            Key                                             Value
0      MS_73418  (0.00033344054827466607, 0.00010276255488861352)
1      MS_13702                      (5.326407062966609e-06, 0.0)
2      MS_22535  (2.8235990612301975e-07, 1.1944192010560073e-05)
3      MS_13236   (4.261186745679879e-07, 1.9199718204188088e-11)
4      MS_10779                     (0.0, 0.00010276255488861352)
...         ...                                               ...
12177  MS_37154                     (0.0, 3.0044968752918066e-06)
12178   MS_3691                      (0.0, 1.097685526474379e-05)
12179  MS_73176                      (0.0, 7.605468272231519e-05)
12180  MS_27134                      (0.0, 5.667965297107003e-07)
12181  MS_17391  (0.00010001519694924355, 1.1230114978388883e-05)

[12182 rows x 2 columns]


In [43]:
v = g_spike_spike.vertex("USER")
neighbour_names = v.neighbours.name.collect()
print(neighbour_names)

['MS_27239', 'MS_15934', 'MS_30441', 'MS_50519', 'MS_54249', 'MS_53154', 'MS_8099', 'MS_42200', 'MS_70289', 'MS_70299', 'MS_65043', 'MS_63670', 'MS_43781', 'MS_67211', 'MS_41385', 'MS_71432', 'MS_3386', 'MS_29370', 'MS_9570', 'MS_22187', 'MS_65235', 'MS_17117', 'MS_56394', 'MS_63434', 'MS_67860', 'MS_39324', 'MS_15296', 'MS_37575', 'MS_25190', 'MS_26075', 'MS_57018', 'MS_61128', 'MS_29844', 'MS_5075', 'MS_58017', 'MS_71138', 'MS_72966', 'MS_56298', 'MS_67455', 'MS_70930', 'MS_57649', 'MS_55290', 'MS_71712', 'MS_5449', 'MS_69638', 'MS_21451', 'MS_62414', 'MS_22997', 'MS_60842', 'MS_29435', 'MS_73326', 'MS_5031', 'MS_64287', 'MS_60350', 'MS_51682', 'MS_15284', 'MS_53926', 'MS_69065', 'MS_19881', 'MS_71990', 'MS_48488', 'MS_27747', 'MS_42683', 'MS_69576', 'MS_15131', 'MS_62847', 'MS_60167', 'MS_19017', 'MS_50178', 'MS_25633', 'MS_52678', 'MS_19630', 'MS_2000', 'MS_46970', 'MS_48689', 'MS_62344', 'MS_64726', 'MS_50735', 'MS_66427', 'MS_66495', 'MS_27021', 'MS_18330', 'MS_63630', 'MS_69708'

In [38]:
# Retrieve top 5 authorities and hubs
top_authorities = hits.top_k(5, reverse=True)
top_hubs = hits.top_k(5, reverse=False)

print("Top 5 Authorities:", top_authorities)
print("Top 5 Hubs:", top_hubs)

Top 5 Authorities: [('MS_45285', (0.000996049609966576, 0.0007854372961446643)), ('MS_15934', (0.0009842063300311565, 0.0005363253294490278)), ('MS_8811', (0.0009040176519192755, 0.0007932359003461897)), ('MS_60349', (0.0008734890725463629, 0.0005140558932907879)), ('MS_51052', (0.0008686163346283138, 0.00042101138387806714))]
Top 5 Hubs: [('MS_30264', (0.0, 1.562382229374816e-36)), ('MS_15825', (0.0, 1.562382229374816e-36)), ('MS_36062', (0.0, 1.562382229374816e-36)), ('MS_57253', (0.0, 1.562382229374816e-36)), ('MS_71519', (0.0, 1.562382229374816e-36))]


In [31]:
segment_results = []

for windowed_graph in g.rolling(window=30000):
    result = rp.hits(windowed_graph, iter_count=20)
    window = 0
    window += 30000
    segment_results.append((window, result))

# Analyse changes in top authorities/hubs over time
for start_time, end_time, result in segment_results:
    print(f"Time Segment: {start_time} - {end_time}")
    print("Top Authorities:", result.top_k(3).to_string())
    print("Top Hubs:", result.top_k(3, reverse=True).to_string())
    print("\n")

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
hubs, authorities = hits.get_all_with_names()

# Find services that are hubs but not strong authorities
potential_bottlenecks = {name: score for name, score in hubs.items() if name not in authorities or authorities[name] < threshold}

print("Potential Bottlenecks:", potential_bottlenecks)

In [14]:
# usage in the function call
balance_result = rp.degree_centrality(g_spike_spike, threads=2)

top_20 = balance_result.top_k(20)
for rank, (name,score) in enumerate(top_20, 1):
    print(f"Rank {rank}: {name} with a score of {score:.5f}")

Rank 1: MS_27421 with a score of 1.00000
Rank 2: USER with a score of 0.72307
Rank 3: MS_37691 with a score of 0.47714
Rank 4: MS_2827 with a score of 0.31582
Rank 5: MS_46825 with a score of 0.28271
Rank 6: MS_21883 with a score of 0.13295
Rank 7: MS_14728 with a score of 0.11823
Rank 8: MS_35973 with a score of 0.10300
Rank 9: MS_44585 with a score of 0.10247
Rank 10: MS_27239 with a score of 0.10089
Rank 11: MS_68453 with a score of 0.09879
Rank 12: MS_46708 with a score of 0.09721
Rank 13: MS_21537 with a score of 0.09564
Rank 14: MS_45285 with a score of 0.09038
Rank 15: MS_25557 with a score of 0.08986
Rank 16: MS_14304 with a score of 0.08881
Rank 17: MS_39211 with a score of 0.08460
Rank 18: MS_32402 with a score of 0.07830
Rank 19: MS_19585 with a score of 0.07725
Rank 20: MS_55085 with a score of 0.07725


In [48]:
for names in neighbour_names:
    for (name,score) in top_20:
        if name in names:
            print(f"Front-end ms: {name}")

Front-end ms: MS_27239
Front-end ms: MS_39211
Front-end ms: MS_25557


In [50]:
print("MS_27421")
v = g_spike_spike.vertex("MS_27421")
neighbour_names = v.neighbours.name.collect()
print(neighbour_names)

MS_27421
['MS_51052', 'MS_15934', 'MS_30441', 'MS_50519', 'MS_56170', 'MS_65240', 'MS_62621', 'MS_46402', 'MS_28081', 'MS_68841', 'MS_48195', 'MS_29370', 'MS_58423', 'MS_56394', 'MS_63434', 'MS_65112', 'MS_36010', 'MS_54030', 'MS_67488', 'MS_37287', 'MS_17995', 'MS_44845', 'MS_49675', 'MS_61128', 'MS_57682', 'MS_25635', 'MS_27283', 'MS_28911', 'MS_56298', 'MS_14698', 'MS_64490', 'MS_71712', 'MS_69638', 'MS_59053', 'MS_36381', 'MS_71844', 'MS_14215', 'MS_56652', 'MS_52086', 'MS_37501', 'MS_37363', 'MS_6739', 'MS_52434', 'MS_25429', 'MS_13312', 'MS_56047', 'MS_71990', 'MS_22682', 'MS_47544', 'MS_68650', 'MS_14902', 'MS_62847', 'MS_60167', 'MS_26234', 'MS_56909', 'MS_9400', 'MS_2000', 'MS_3311', 'MS_17979', 'MS_63627', 'MS_27021', 'MS_63630', 'MS_7932', 'MS_22184', 'MS_61034', 'MS_68299', 'MS_72533', 'MS_47345', 'MS_65823', 'MS_46799', 'MS_17639', 'MS_2109', 'MS_35733', 'MS_39472', 'MS_34796', 'MS_61676', 'MS_69823', 'MS_48334', 'MS_36601', 'MS_19960', 'MS_20896', 'MS_19709', 'MS_34151', 

In [51]:
for names in neighbour_names:
    for (name,score) in top_20:
        if name in names:
            print(f"Front-end ms: {name}")

Front-end ms: MS_45285
